<a href="https://colab.research.google.com/github/FatLads/Notebooks/blob/main/FlatLand_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install -y xvfb ffmpeg
!pip install -q 'imageio==2.4.0'
!pip install -q pyvirtualdisplay
!pip install -q tf-agents
!pip install -q flatland-rl

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 784 kB of archives.
After this operation, 2,270 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.8 [784 kB]
Fetched 784 kB in 1s (1,099 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected pa

In [ ]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment, py_environment
from tf_agents.eval import metric_utils
from tf_agents.networks import sequential 
from tf_agents.policies import random_tf_policy 
from tf_agents.environments import utils
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory, time_step
from tf_agents.specs import tensor_spec, BoundedArraySpec
from tf_agents.utils import common 
from flatland.envs.rail_env import RailEnv

display = pyvirtualdisplay.Display(visible=0, size=(1400,900)).start()

In [ ]:
num_iterations = 30000 # @param {type:"integer"}

initial_collect_steps = 100 # @param {type:"integer"}
collect_steps_per_iteration = 1 # @param {type:"integer"}
replay_buffer_max_length = 100000 # @param {type:"integer"}

batch_size = 64 # @param {type:"integer"}
learning_rate = 1e-4 # @param {type:"number"}
log_interval = 200 # @param {type:"integer"}

num_eval_episodes = 10 # @param {type:"integer"}
eval_interval = 1000 # @param {type:"integer"}

Next, we're gonna try and reimplement the `Rails` env with all of the things that TensorFlow needs, like explained [here](https://www.tensorflow.org/agents/tutorials/2_environments_tutorial) 

In [81]:
class FatRails(py_environment.PyEnvironment):
    def __init__(self, width, height):
        self.env = RailEnv(width, height)
        self.width = width
        self.height = height
    def action_spec(self):
        return BoundedArraySpec(shape=(), dtype=np.int64, name='action', minimum=0, maximum=4) # We need to convert from an array of actions (index=agent, value=action) to a dict
    def observation_spec(self):
        return BoundedArraySpec(shape=(self.width, self.height, 16), dtype=np.int32, minimum=0, maximum=1, name='observation') #TODO
    def _step(self, action):
        action_dict = {v: k for v, k in enumerate(action)}
        step_env = self.env.step(action_dict)
        return time_step.transition(step_env[0][0][0], step_env[1][0])
    def _reset(self):
        reset = self.env.reset()
        return time_step.restart(np.array(reset[0][0][0], dtype=np.int32))

In [79]:
environment = FatRails(16,16)
environment._reset()
environment._step([0])

TimeStep(step_type=array(1, dtype=int32), reward=array(-1., dtype=float32), discount=array(1., dtype=float32), observation=array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 1., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 1., 0., 0.],
        [1., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 1., 0.],
        [1., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 1., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],


In [ ]:
from tf_agents.specs import BoundedArraySpec
env = RailEnv(width=16, height=16)
# TODO: The self.number_of_agents spec might be wrong: it is a dict, not a ndarray

In [ ]:
from flatland.envs.rail_env import RailEnvActions
env.step({0: RailEnvActions.DO_NOTHING})[1]

NameError: ignored

In [ ]:
fc_layer_params = (100,50)

action_tensor_spec = tensor_spec.from_spec(env.action_spec)
num_actions = action_tensor_spec.maximum-action_tensor_spec.minimum + 1



def dense_layer(num_units):
    return tf.keras.layers.Dense(
        num_units,
        activation=tf.keras.activations.relu,
        kernel_initializer=tf.keras.initializers.VarianceScaling(
            scale=2.0, mode='fan_in', distribution='truncated_normal'
        )
    )

dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03
    ),
    bias_initializer=tf.keras.initializers.Constant(-0.2)
)
q_net = sequential.Sequential(dense_layers+[q_values_layer])

In [ ]:
env.reset()

In [ ]:
from tf_agents.trajectories.time_step import TimeStep
time_step_spec = TimeStep()